<a href="https://colab.research.google.com/github/joseph7104/-1INF46-Plan_Compras_Produccion/blob/master/notebooks/benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!git clone https://github.com/joseph7104/-1INF46-Plan_Compras_Produccion.git
%cd "./-1INF46-Plan_Compras_Produccion"

Cloning into '-1INF46-Plan_Compras_Produccion'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 45 (delta 7), reused 33 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 4.02 MiB | 14.48 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/-1INF46-Plan_Compras_Produccion


# **Setup y carga del dataset**

In [9]:
!pip install pmdarima prophet xgboost scikit-learn --quiet
!pip install -q "numpy>=2.1,<2.3" statsmodels --upgra
import pandas as pd, numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

ambiguous option: --upgra (--upgrade, --upgrade-strategy?)


In [10]:
PROC_DIR = "data/processed"
df = pd.read_csv(f"{PROC_DIR}/dataset_forecast_diario_features.csv", parse_dates=['fecha'])
platos = df['plato'].unique().tolist()


# **Benchmark**

## **Arima**

In [11]:
# ⚠️ Esto reiniciará el runtime de Colab
!pip install -q "numpy==1.26.4" "pmdarima==2.0.4"

import os, sys, time
os.kill(os.getpid(), 9)  # fuerza reinicio del kernel


ERROR: Operation cancelled by user


KeyboardInterrupt: 

In [12]:
import pmdarima as pm
print(pm.__version__)  # debería mostrar 2.0.4


2.0.4


In [13]:
import pmdarima as pm

res_arima = []
for p in platos:
    d = df[df.plato==p].sort_values('fecha')
    y = d['cantidad'].values
    split = int(len(y)*0.8)
    y_tr, y_te = y[:split], y[split:]

    m = pm.auto_arima(y_tr, seasonal=True, m=7, stepwise=True, suppress_warnings=True)
    y_hat = m.predict(n_periods=len(y_te))

    mae = mean_absolute_error(y_te, y_hat)
    rmse = np.sqrt(((y_te - y_hat)**2).mean())
    res_arima.append([p, mae, rmse])


/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12

## **Prophet**

In [14]:
from prophet import Prophet

res_prophet = []
reg_cols = [c for c in df.columns if c.startswith('evento_') or c.startswith('clima_')]

for p in platos:
    d = df[df.plato==p].sort_values('fecha')
    data = d[['fecha','cantidad']].rename(columns={'fecha':'ds','cantidad':'y'})
    m = Prophet(weekly_seasonality=True, yearly_seasonality=True, daily_seasonality=False)
    for c in reg_cols: m.add_regressor(c)

    train_idx = int(len(d)*0.8)
    train = pd.concat([data.iloc[:train_idx], d[reg_cols].iloc[:train_idx]], axis=1)
    test  = pd.concat([data.iloc[train_idx:], d[reg_cols].iloc[train_idx:]], axis=1)

    m.fit(train)
    y_hat = m.predict(test[['ds']+reg_cols])['yhat'].values
    y_te  = test['y'].values
    mae = mean_absolute_error(y_te, y_hat)
    rmse = np.sqrt(((y_te - y_hat)**2).mean())
    res_prophet.append([p, mae, rmse])


DEBUG:cmdstanpy:input tempfile: /tmp/tmp45t46c42/fqsj1fo8.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp45t46c42/xos0x1cs.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.12/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=83175', 'data', 'file=/tmp/tmp45t46c42/fqsj1fo8.json', 'init=/tmp/tmp45t46c42/xos0x1cs.json', 'output', 'file=/tmp/tmp45t46c42/prophet_modely_bxtaum/prophet_model-20251013074218.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:42:18 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:42:18 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmp45t46c42/a8b9zymx.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp45t46c42/hmsup76t.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

## **XGBoost**

In [15]:
from xgboost import XGBRegressor

X_cols = [c for c in df.columns if c not in ['fecha','plato','cantidad','monto_total']]
res_xgb = []

for p in platos:
    d = df[df.plato==p].sort_values('fecha')
    X, y = d[X_cols], d['cantidad']
    split = int(len(X)*0.8)
    X_tr, X_te = X.iloc[:split], X.iloc[split:]
    y_tr, y_te = y.iloc[:split], y.iloc[split:]

    model = XGBRegressor(n_estimators=400, learning_rate=0.05, max_depth=6,
                         subsample=0.8, colsample_bytree=0.8, random_state=42)
    model.fit(X_tr, y_tr)
    y_hat = model.predict(X_te)

    mae = mean_absolute_error(y_te, y_hat)
    rmse = np.sqrt(((y_te - y_hat)**2).mean())
    res_xgb.append([p, mae, rmse])


# **Comparativo**

In [19]:
import pandas as pd, matplotlib.pyplot as plt
import os

# Create the directory if it doesn't exist
FIG_DIR = "reports/figures"
os.makedirs(FIG_DIR, exist_ok=True)


res = (pd.DataFrame(res_arima, columns=['plato','MAE_ARIMA','RMSE_ARIMA'])
         .merge(pd.DataFrame(res_prophet, columns=['plato','MAE_PROPHET','RMSE_PROPHET']), on='plato')
         .merge(pd.DataFrame(res_xgb, columns=['plato','MAE_XGB','RMSE_XGB']), on='plato'))
display(res.sort_values('RMSE_XGB'))

# Generate and save plots for each dish (XGB)
for p in platos:
    d = df[df.plato==p].sort_values('fecha')
    split = int(len(d)*0.8)
    X = d[X_cols]; y = d['cantidad']
    model = XGBRegressor(n_estimators=400, learning_rate=0.05, max_depth=6,
                         subsample=0.8, colsample_bytree=0.8, random_state=42)
    model.fit(X.iloc[:split], y.iloc[:split])
    y_hat = model.predict(X.iloc[split:])

    plt.figure(figsize=(12,4))
    plt.plot(d['fecha'].iloc[split:], y.iloc[split:].values, label='Real')
    plt.plot(d['fecha'].iloc[split:], y_hat, label='Predicho')
    plt.title(f'Real vs Predicho — {p} (XGB)')
    plt.legend()
    plt.savefig(f"{FIG_DIR}/{p.replace(' ', '_')}_xgb_prediction.png") # Save the figure
    plt.close() # Close the figure to free up memory

print(f"✅ Gráficos guardados en {FIG_DIR}")

,plato,MAE_ARIMA,RMSE_ARIMA,MAE_PROPHET,RMSE_PROPHET,MAE_XGB,RMSE_XGB
4,Tallarines Verdes,5.668789,7.063286,5.465653,6.642905,5.668951,6.945568
0,Arroz Chaufa,7.860599,9.731898,7.471529,9.115306,7.911308,9.873842
1,Ceviche Clásico,10.326326,12.946663,9.691183,11.827565,10.277339,12.412126
3,Pollo a la Brasa,12.200675,14.913290,11.535297,13.858997,11.759839,14.359419
2,Lomo Saltado,14.515604,17.835035,13.882061,16.590336,13.941113,17.148761


✅ Gráficos guardados en reports/figures


In [18]:
import os
MET_DIR = "reports/metrics"
os.makedirs(MET_DIR, exist_ok=True)
res.to_csv(f"{MET_DIR}/benchmark_modelos.csv", index=False)
print("✅ Métricas guardadas en reports/metrics/benchmark_modelos.csv")

✅ Métricas guardadas en reports/metrics/benchmark_modelos.csv


In [21]:
# Calculate the mean metrics across all dishes for each model
mean_metrics = res[['MAE_ARIMA', 'RMSE_ARIMA', 'MAE_PROPHET', 'RMSE_PROPHET', 'MAE_XGB', 'RMSE_XGB']].mean().reset_index()
mean_metrics.columns = ['Metric', 'Average Value']

# Reshape the table for better readability
mean_metrics['Model'] = mean_metrics['Metric'].apply(lambda x: x.split('_')[1])
mean_metrics['Metric Type'] = mean_metrics['Metric'].apply(lambda x: x.split('_')[0])
mean_metrics_pivot = mean_metrics.pivot_table(index='Model', columns='Metric Type', values='Average Value')

print("Tabla de Benchmark General (Métricas Promedio por Modelo):")
display(mean_metrics_pivot)

# Optionally save this table to a CSV as well
mean_metrics_pivot.to_csv(f"{MET_DIR}/benchmark_general_average_metrics.csv")
print(f"✅ Tabla de benchmark general (métricas promedio) guardada en {MET_DIR}/benchmark_general_average_metrics.csv")

Tabla de Benchmark General (Métricas Promedio por Modelo):


Metric Type,MAE,RMSE
Model,,
ARIMA,10.114399,12.498034
PROPHET,9.609144,11.607022
XGB,9.911710,12.147943


✅ Tabla de benchmark general (métricas promedio) guardada en reports/metrics/benchmark_general_average_metrics.csv


In [20]:
import pandas as pd
import os

MET_DIR = "reports/metrics"
os.makedirs(MET_DIR, exist_ok=True)

# Display the benchmark table
print("Tabla de Benchmark General:")
display(res.sort_values('RMSE_XGB'))

# Save the benchmark table to a CSV file
res.to_csv(f"{MET_DIR}/benchmark_general.csv", index=False)
print(f"✅ Tabla de benchmark general guardada en {MET_DIR}/benchmark_general.csv")

Tabla de Benchmark General:


,plato,MAE_ARIMA,RMSE_ARIMA,MAE_PROPHET,RMSE_PROPHET,MAE_XGB,RMSE_XGB
4,Tallarines Verdes,5.668789,7.063286,5.465653,6.642905,5.668951,6.945568
0,Arroz Chaufa,7.860599,9.731898,7.471529,9.115306,7.911308,9.873842
1,Ceviche Clásico,10.326326,12.946663,9.691183,11.827565,10.277339,12.412126
3,Pollo a la Brasa,12.200675,14.913290,11.535297,13.858997,11.759839,14.359419
2,Lomo Saltado,14.515604,17.835035,13.882061,16.590336,13.941113,17.148761


✅ Tabla de benchmark general guardada en reports/metrics/benchmark_general.csv


In [22]:
# ======================================================================
# PASO 0: CONFIGURAR GIT (Por si el entorno se reinició)
# ======================================================================
!git config --global user.email "joseph7104@gmail.com"
!git config --global user.name "joseph7104"

# ======================================================================
# PASO 1: AÑADIR LOS ARCHIVOS "UNTRACKED"
# Este es el comando clave que faltaba por ejecutarse correctamente.
# Le dice a Git que rastree estos archivos.
# ======================================================================
!git add data/processed/dataset_forecast_diario_features.csv reports/figures/*.png reports/metrics/baseline_metrics.csv

# ======================================================================
# PASO 2: CREAR EL COMMIT
# Ahora que los archivos están añadidos, el commit funcionará.
# ======================================================================
!git commit -m "Añadir resultados de EDA, features y métricas baseline"

# ======================================================================
# PASO 3: SUBIR A GITHUB (PUSH)
# Este código ya lo tenías bien.
# ======================================================================
from google.colab import userdata

# --- ¡CONFIGURA ESTO! ---
username = "joseph7104"
repository = "-1INF46-Plan_Compras_Produccion" # Ya está bien configurado
# -------------------------

try:
    token = userdata.get('GH_TOKEN')
except userdata.SecretNotFoundError:
    print("Error: El 'secret' con nombre 'GH_TOKEN' no se encontró.")
    raise

repo_url_with_token = f"https://{token}@github.com/{username}/{repository}.git"
!git remote set-url origin {repo_url_with_token}

print("\nSubiendo cambios a GitHub...")
!git push origin master

[master eaee367] Añadir resultados de EDA, features y métricas baseline
 5 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 reports/figures/Arroz_Chaufa_xgb_prediction.png
 create mode 100644 "reports/figures/Ceviche_Cl\303\241sico_xgb_prediction.png"
 create mode 100644 reports/figures/Lomo_Saltado_xgb_prediction.png
 create mode 100644 reports/figures/Pollo_a_la_Brasa_xgb_prediction.png
 create mode 100644 reports/figures/Tallarines_Verdes_xgb_prediction.png

Subiendo cambios a GitHub...
Enumerating objects: 12, done.
Counting objects: 100% (12/12), done.
Delta compression using up to 2 threads
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 605.60 KiB | 16.37 MiB/s, done.
Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/joseph7104/-1INF46-Plan_Compras_Produccion.git
   6644657..eaee367  master -> master
